# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 22 2022 9:30AM,244635,20,8376177-BO,"ACI Healthcare USA, Inc.",Released
1,Aug 22 2022 9:30AM,244635,20,8386339-BO,"ACI Healthcare USA, Inc.",Released
2,Aug 22 2022 9:30AM,244635,20,8386324-BO,"ACI Healthcare USA, Inc.",Released
3,Aug 22 2022 9:30AM,244635,20,8386318-BO,"ACI Healthcare USA, Inc.",Released
4,Aug 22 2022 9:30AM,244635,20,8395074-BO,"ACI Healthcare USA, Inc.",Released
5,Aug 22 2022 9:30AM,244635,20,8418627,"ACI Healthcare USA, Inc.",Released
6,Aug 22 2022 9:30AM,244635,20,8419182,"ACI Healthcare USA, Inc.",Released
7,Aug 22 2022 9:30AM,244635,20,8419770,"ACI Healthcare USA, Inc.",Released
8,Aug 22 2022 9:30AM,244635,20,8419774,"ACI Healthcare USA, Inc.",Released
9,Aug 22 2022 9:30AM,244635,20,8422390,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,244632,Released,8
15,244633,Released,1
16,244634,Released,41
17,244635,Released,10
18,244636,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244632,NaN,8.0
244633,NaN,1.0
244634,NaN,41.0
244635,NaN,10.0
244636,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244604,6.0,0.0
244609,0.0,15.0
244610,0.0,10.0
244611,0.0,1.0
244613,21.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244604,6,0
244609,0,15
244610,0,10
244611,0,1
244613,21,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244604,6,0
1,244609,0,15
2,244610,0,10
3,244611,0,1
4,244613,21,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244604,6,
1,244609,,15
2,244610,,10
3,244611,,1
4,244613,21,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc."
10,Aug 22 2022 9:24AM,244633,10,Bio-PRF
11,Aug 22 2022 9:24AM,244636,12,Hush Hush
12,Aug 22 2022 9:24AM,244634,12,Hush Hush
53,Aug 22 2022 9:23AM,244632,10,Bio-PRF
61,Aug 22 2022 8:39AM,244628,16,Sartorius Bioprocess Solutions
62,Aug 22 2022 8:37AM,244627,16,Sartorius Lab Products and Service
63,Aug 22 2022 8:37AM,244627,16,Sartorius Bioprocess Solutions
65,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation
71,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc.",,10
1,Aug 22 2022 9:24AM,244633,10,Bio-PRF,,1
2,Aug 22 2022 9:24AM,244636,12,Hush Hush,,1
3,Aug 22 2022 9:24AM,244634,12,Hush Hush,,41
4,Aug 22 2022 9:23AM,244632,10,Bio-PRF,,8
5,Aug 22 2022 8:39AM,244628,16,Sartorius Bioprocess Solutions,,1
6,Aug 22 2022 8:37AM,244627,16,Sartorius Lab Products and Service,,3
7,Aug 22 2022 8:37AM,244627,16,Sartorius Bioprocess Solutions,,3
8,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,6,
9,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc.",10,
1,Aug 22 2022 9:24AM,244633,10,Bio-PRF,1,
2,Aug 22 2022 9:24AM,244636,12,Hush Hush,1,
3,Aug 22 2022 9:24AM,244634,12,Hush Hush,41,
4,Aug 22 2022 9:23AM,244632,10,Bio-PRF,8,
5,Aug 22 2022 8:39AM,244628,16,Sartorius Bioprocess Solutions,1,
6,Aug 22 2022 8:37AM,244627,16,Sartorius Lab Products and Service,3,
7,Aug 22 2022 8:37AM,244627,16,Sartorius Bioprocess Solutions,3,
8,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,,6
9,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,15,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc.",10,
1,Aug 22 2022 9:24AM,244633,10,Bio-PRF,1,
2,Aug 22 2022 9:24AM,244636,12,Hush Hush,1,
3,Aug 22 2022 9:24AM,244634,12,Hush Hush,41,
4,Aug 22 2022 9:23AM,244632,10,Bio-PRF,8,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc.",10.0,NaN
1,Aug 22 2022 9:24AM,244633,10,Bio-PRF,1.0,NaN
2,Aug 22 2022 9:24AM,244636,12,Hush Hush,1.0,NaN
3,Aug 22 2022 9:24AM,244634,12,Hush Hush,41.0,NaN
4,Aug 22 2022 9:23AM,244632,10,Bio-PRF,8.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 22 2022 9:30AM,244635,20,"ACI Healthcare USA, Inc.",10.0,0.0
1,Aug 22 2022 9:24AM,244633,10,Bio-PRF,1.0,0.0
2,Aug 22 2022 9:24AM,244636,12,Hush Hush,1.0,0.0
3,Aug 22 2022 9:24AM,244634,12,Hush Hush,41.0,0.0
4,Aug 22 2022 9:23AM,244632,10,Bio-PRF,8.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2446186,51.0,6.0
12,489270,42.0,0.0
16,733882,7.0,0.0
19,489227,10.0,24.0
20,489253,28.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2446186,51.0,6.0
1,12,489270,42.0,0.0
2,16,733882,7.0,0.0
3,19,489227,10.0,24.0
4,20,489253,28.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,51.0,6.0
1,12,42.0,0.0
2,16,7.0,0.0
3,19,10.0,24.0
4,20,28.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,12,Released,42.0
2,16,Released,7.0
3,19,Released,10.0
4,20,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,6.0,0.0,0.0,24.0,0.0
Released,51.0,42.0,7.0,10.0,28.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,6.0,0.0,0.0,24.0,0.0
1,Released,51.0,42.0,7.0,10.0,28.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,6.0,0.0,0.0,24.0,0.0
1,Released,51.0,42.0,7.0,10.0,28.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()